In [1]:
import json
from dataclasses import asdict

from openai_dm.character_sheet import Character

with open("char_sheet.json", "w") as f:
    json.dump(asdict(Character()), f)

with open("char_sheet.json", "r") as f:
    char_sheet = json.load(f)
    print(char_sheet["race"])
    print(char_sheet["ability_scores"])

None
{'strength': 8, 'dexterity': 8, 'constitution': 8, 'intelligence': 8, 'wisdom': 8, 'charisma': 8}


In [2]:
import json
from dotenv import load_dotenv
from dataclasses import asdict
from attr import field, define, Factory
import random
from griptape.artifacts import TextArtifact
from griptape.memory.tool import TextToolMemory
from griptape.tools import BaseTool, FileManager
from griptape.tasks import PromptTask, ToolkitTask
from griptape.utils.decorators import activity
from schema import Schema, Literal, Optional
from griptape.drivers import OpenAiChatPromptDriver
from griptape.tasks import PromptTask
from griptape.structures import Pipeline

from openai_dm.tools import CharacterSheetUpdater, RaceChanger
from openai_dm.character_sheet import Character

load_dotenv()

character_memory = TextToolMemory(
    query_engine = field(kw_only=True),
    summary_engine = field(kw_only=True, default=Factory(lambda : PromptSummaryEngine()))
)

file_manager = FileManager(
    #input_memory=[text_memory],
    # output_memory={
    #     "load_files_from_disk": [character_memory]
    # }
)

driver = OpenAiChatPromptDriver(
    model="gpt-4",
    max_tokens=500 # you can experiment with the number of tokens
)


pipeline = Pipeline(
    #memory=ConversationMemory(),
)

pipeline.add_tasks(
    PromptTask('''
        The user said {{ args[0] }}. Respond with a json of the ability score bonuses for the chosen race.
        Example 1: I'll be a halfling
        {"ability_scores": {"dexterity": 2}}
        Example 2: I want to play as a human.
        {"ability_scores": {"strength": 1, "constitution": 1, "dexterity": 1, "intelligence": 1, "wisdom": 1, "charisma": 1}}''',
        prompt_driver=driver,
    ),
    ToolkitTask(
        '''Update the character sheet located at char_sheet.json with:
            new_vals as {{ parent_output }}
        Save the result as updated_char_sheet.json
        ''', 
        tools = [file_manager, CharacterSheetUpdater()],
        prompt_driver=driver,
    ),
)
output = pipeline.run("I'll be a tiefling")

[09/30/23 21:06:32] INFO     PromptTask 69927ed5afbb45d98003828e21aa17a8                                           
                             Input:                                                                                
                                     The user said I'll be a tiefling. Respond with a json of the ability score    
                             bonuses for the chosen race.                                                          
                                     Example 1: I'll be a halfling                                                 
                                     {"ability_scores": {"dexterity": 2}}                                          
                                     Example 2: I want to play as a human.                                         
                                     {"ability_scores": {"strength": 1, "constitution": 1, "dexterity": 1,         
                             "intelligence": 1, "wisdom": 1, "charisma": 1}}

[09/30/23 21:06:33] INFO     PromptTask 69927ed5afbb45d98003828e21aa17a8                                           
                             Output: {"ability_scores": {"intelligence": 1, "charisma": 2}}

                    INFO     ToolkitTask 4d2a56df16d84b57aa04b2f04eb782e9                                          
                             Input: Update the character sheet located at char_sheet.json with:                    
                                         new_vals as {"ability_scores": {"intelligence": 1, "charisma": 2}}        
                                     Save the result as updated_char_sheet.json                                    
                    

[09/30/23 21:06:41] INFO     Subtask c9320d5139eb4ffa9244451aaf50bea2                                              
                             Thought: The first step is to load the character sheet from the specified file. I will
                             use the FileManager tool with the load_files_from_disk activity for this.             
                             Action: {"type": "tool", "name": "FileManager", "activity": "load_files_from_disk",   
                             "input": {"values": {"paths": ["char_sheet.json"]}}}

                    INFO     Subtask c9320d5139eb4ffa9244451aaf50bea2                                              
                             Observation: b'{"name": null, "class_": null, "subclasse": null, "level": 1, "race":  
                             null, "alignment": null, "background": null, "proficiency_bonus": 2, "ability_scores":
                             {"strength": 8, "dexterity": 8, "constitution": 8, "intelligence": 8, "wisdom": 8,    
                             "charisma": 8}, "saving_throw_proficiencies": {"strength": false, "dexterity": false, 
                             "constitution": false, "intelligence": false, "wisdom": false, "charisma": false},    
                             "skill_proficiencies": {"acrobatics": false, "animal_handling": false, "arcana":      
                             false, "athletics": false, "deception": false, "history": false, "insight": false,    
                             "intimidation": false, "investigation": false, "medicine": false, "nature": false,    
                             "perception": false, "performance": false, "persuasion": false, "religion": false,    
                             "sleight_of_hand": false, "stealth": false, "survival": false}, "AC": 10, "speed": 30,
                             "hit_point_max": 6}'

[09/30/23 21:07:07] INFO     Subtask 71f8a13517404ebb9dc84df5a2c860ab                                              
                             Thought: Now that I have the character sheet, I can use the CharacterSheetUpdater tool
                             with the update_character_sheet activity to update the ability scores.                
                             Action: {"type": "tool", "name": "CharacterSheetUpdater", "activity":                 
                             "update_character_sheet", "input": {"values": {"new_vals": "{\"ability_scores\":      
                             {\"intelligence\": 1, \"charisma\": 2}}", "character_sheet": "{\"name\": null,        
                             \"class_\": null, \"subclasse\": null, \"level\": 1, \"race\": null, \"alignment\":   
                             null, \"background\": null, \"proficiency_bonus\": 2, \"ability_scores\":             
                             {\"strength\": 8, \"dexterity\": 8, \"constitution\": 8, \"intelligence\": 8,         
                             \"wisdom\": 8, \"charisma\": 8}, \"saving_throw_proficiencies\": {\"strength\": false,
                             \"dexterity\": false, \"constitution\": false, \"intelligence\": false, \"wisdom\":   
                             false, \"charisma\": false}, \"skill_proficiencies\": {\"acrobatics\": false,         
                             \"animal_handling\": false, \"arcana\": false, \"athletics\": false, \"deception\":   
                             false, \"history\": false, \"insight\": false, \"intimidation\": false,               
                             \"investigation\": false, \"medicine\": false, \"nature\": false, \"perception\":     
                             false, \"performance\": false, \"persuasion\": false, \"religion\": false,            
                             \"sleight_of_hand\": false, \"stealth\": false, \"survival\": false}, \"AC\": 10,     
                             \"speed\": 30, \"hit_point_max\": 6}"}}}

[09/30/23 21:07:08] INFO     Subtask 71f8a13517404ebb9dc84df5a2c860ab                                              
                             Observation: Output of "CharacterSheetUpdater.update_character_sheet" was stored in   
                             memory with memory_name "TextToolMemory" and artifact_namespace                       
                             "06e910c529ad4fc598757b56c1ec3362"

[09/30/23 21:07:16] INFO     Subtask 2f2c0753c71c45cab4166fa8fea059dd                                              
                             Thought: The updated character sheet is stored in memory. I need to retrieve it from  
                             memory and save it to the specified file. I will use the FileManager tool with the    
                             save_memory_artifacts_to_disk activity for this.                                      
                             Action: {"type": "tool", "name": "FileManager", "activity":                           
                             "save_memory_artifacts_to_disk", "input": {"values": {"dir_name": ".", "file_name":   
                             "updated_char_sheet.json", "memory_name": "TextToolMemory", "artifact_namespace":     
                             "06e910c529ad4fc598757b56c1ec3362"}}}

                    INFO     Subtask 2f2c0753c71c45cab4166fa8fea059dd                                              
                             Observation: saved successfully

[09/30/23 21:07:18] INFO     ToolkitTask 4d2a56df16d84b57aa04b2f04eb782e9                                          
                             Output: The character sheet has been successfully updated and saved as                
                             updated_char_sheet.json.

In [5]:
import json

with open("updated_char_sheet.json", "r") as f:
    sheet = json.load(f)
    print(sheet["ability_scores"])

{'strength': 8, 'dexterity': 8, 'constitution': 8, 'intelligence': 9, 'wisdom': 8, 'charisma': 10}
